# Implementasi Gaussian Naive Bayes

Dalam praktik kali ini, kita akan melakukan praktik untuk menerapkan algorithm gaussian naive Bayes untuk memodelkan data klasifikasi jenis narkoba yang dikonsumis oleh seseorang berdasarkan usia, gender, tekanan darah, tingkat kolesterol dalam darah, dan kandungan ratio kandungan natrium terhadap kalium dalam darah.

Di bawah ini akan dijelaskan secara rinci pemodelan yang dilakukan dengan beberapa proses yang dilalui untuk mendapatkan hasil dengan score model yang paling baik.

## 1. Mengimport Beberapa Package Utama dalam Pemodelan

Pada bagian ini, kita akan melakukan pengimportan beberapa package utama yang akan digunakan dalam mengolah data dan melakukan pemodelan pada data.

In [1]:
print(1)

1


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer, LabelEncoder
from category_encoders import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate

## 2. Melakukan Data Preprocessing

Pada bagian ini, akan dilakukan pemngimportan data dan beberapa proses penting lainnya untuk mengolah data, diantaranya:
+ menganalisis statistik deskriptif dari data
+ mengecek missing value dari data
+ mengahpus kolom yang dirasa tidak penting
+ mendeteksi outlier dalam data dan membersihkannya
+ melakukan pelabelan pada data kategorikal (ordinal dan nominal)
+ rescaling data untuk mengurangi kebiasan yang terjadi

In [3]:
dataset = pd.read_csv("drugs.csv")

In [4]:
dataset.isnull().sum()

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64

In [5]:
dataset.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [6]:
dataset.describe(include="all")

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
count,200.000000,200,200,200,200.000000,200
unique,NaN,2,3,2,NaN,5
top,NaN,M,HIGH,HIGH,NaN,DrugY
freq,NaN,104,77,103,NaN,91
mean,44.315000,NaN,NaN,NaN,16.084485,NaN
std,16.544315,NaN,NaN,NaN,7.223956,NaN
min,15.000000,NaN,NaN,NaN,6.269000,NaN
25%,31.000000,NaN,NaN,NaN,10.445500,NaN
50%,45.000000,NaN,NaN,NaN,13.936500,NaN
75%,58.000000,NaN,NaN,NaN,19.380000,NaN


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [ ]:
plt.figure()
sns.boxplot(x=dataset['Age'])
plt.show()

plt.figure()
sns.boxplot(x=dataset['Na_to_K'])
plt.show()

In [ ]:
Q1 = (dataset[['Na_to_K']]).quantile(0.25)
Q3 = (dataset[['Na_to_K']]).quantile(0.75)

IQR = Q3 - Q1

max_val = Q3 + (1.5 * IQR)
min_val = Q1 - (1.5 * IQR)

batas_atas = (dataset > max_val)
batas_bawah = (dataset < min_val)

data_clean = dataset.mask(batas_bawah, min_val, axis=1)
data_clean = data_clean.mask(batas_atas, max_val, axis=1)

In [ ]:
plt.figure()
sns.boxplot(x=data_clean['Na_to_K'])
plt.show()

In [ ]:
print(data_clean.groupby('BP').count())
print(data_clean.groupby('Cholesterol').count())

In [ ]:
LE = LabelEncoder()
data_clean['Sex'] = LE.fit_transform(data_clean['Sex'])

In [ ]:
dict_bp = [{'col': 'BP', 'mapping': {'LOW': 1, 'NORMAL': 2, 'HIGH': 3}}]

oe_bp = OrdinalEncoder(cols='BP', mapping=dict_bp)
data_clean = oe_bp.fit_transform(data_clean)

In [ ]:
dict_chol = [{'col': 'Cholesterol', 'mapping': {'NORMAL': 1, 'HIGH': 2}}]

oe_chol = OrdinalEncoder(cols='BP', mapping=dict_chol)
data_clean = oe_chol.fit_transform(data_clean)

In [ ]:
data_clean[['Na_to_K']] = PowerTransformer(method='box-cox').fit_transform(data_clean[['Na_to_K']])

In [ ]:
plt.figure()
sns.kdeplot(x=data_clean['Na_to_K'])
plt.show()

## 3. Melakukan Pemodelan

Pada bagian ini, yang akan kita lakukan antara lain:
+ melakukan splitting dataset ke dalam training dan testing dataset
+ membuat model dengan GaussianNB yang ada dalam package sklearn.naive_bayes
+ mengevaluasi model melalui cross-validate model
+ mengecek error dan keakuratan dari model dengan confusion matrix dan classification report

In [ ]:
from yellowbrick.features import Rank2D

X_clean = data_clean.drop('Drug', axis=1)
y_clean = data_clean['Drug']

r2d = Rank2D(method='spearman')
r2d.fit(X_clean, y_clean)
r2d.transform(X_clean)
r2d.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.25, random_state=0)

In [ ]:
from sklearn.naive_bayes import GaussianNB

multi = GaussianNB()
multi.get_params()

In [ ]:
grid = GridSearchCV(multi, param_grid=dict(var_smoothing=np.logspace(-9, -1, 10)), n_jobs=-1)
model = grid.fit(X_train, y_train)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
acc = accuracy_score(prediction, y_test)
print("Score model: {}".format(acc))

In [ ]:
scoring = ['precision_macro', 'recall_macro', 'f1_macro']

scores = cross_validate(model, X_clean, y_clean, cv=5, scoring=scoring)
print('Precision:', scores['test_precision_macro'])
print('Recall:', scores['test_recall_macro'])
print('F1 score:', scores['test_f1_macro'])

In [ ]:
from yellowbrick.classifier import ConfusionMatrix

cm = ConfusionMatrix(model)
cm.fit(X_train, y_train)
cm.score(X_test, y_test)
cm.show()

In [ ]:
from yellowbrick.classifier import ClassificationReport

cr = ClassificationReport(model, support=True)
cr.fit(X_train, y_train)
cr.score(X_test, y_test)
cr.show()